In [1]:
from landmark_detection.pipeline import Pipeline_Landmark_Detection
from benchmark.revisitop.dataset import configdataset
from benchmark.evaluation import run_evaluation2, save_evaluation_result

import os
import pandas as pd

In [2]:
VERSION = 0

force_rebuild = False
save_every = 500
config = {
    'version': VERSION,
    
    # DATABASE
    'min_area': 0.4,
    'min_sim_db': 0.9,
    'keep_full_img': True,
    
    # PIPELINE
    'detector_file': "yolov8n-oiv7.pt",
    'extractor_onnx_file': "cvnet-sg-v" + str(VERSION) + ".onnx",
    'pipeline_onnx_file': "pipeline-yolo-cvnet-sg-v" + str(VERSION) + ".onnx",
    'image_dim': (640, 640),
    'allowed_classes': [41,68,70,74,87,95,113,144,150,158,164,165,193,205,212,224,257,
                                  298,310,335,351,354,390,393,401,403,439,442,457,466,489,510,512,
                                  514,524,530,531,543,546,554,565,573,580,588,591],
    'score_thresh': 0.05,
    'iou_thresh': 0.25,
    'scales': [0.7071, 1.0, 1.4142],
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
    'rgem_pr': 2.5,
    'rgem_size': 5,
    'gem_p': 4.6,
    'sgem_ps': 10.0,
    'sgem_infinity': False,
    'eps': 1e-8,
}
topk = 10
min_sim = 0.70
min_votes = 0.60
remove_inner_boxes = 0.5
join_boxes = True

In [3]:
# Fixed Paths
CLASS_NAMES_PATH = os.path.join("oiv7.yaml")
DATASETS_PATH = os.path.abspath("datasets")
ROXFORD5K_PATH = os.path.join(DATASETS_PATH, "roxford5k", "jpg")
ROXFORD5K_PKL = os.path.join(DATASETS_PATH, "roxford5k", "gnd_roxford5k.pkl")
ROXFORD5K_CSV = os.path.join(DATASETS_PATH, "roxford5k", "roxford5k_image_data.csv")
RESULTS_PATH = os.path.join("benchmark", "results", "results.csv")

In [4]:
cfg = configdataset('roxford5k', DATASETS_PATH)

In [6]:
first_version = 44
score_thresh_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i, score_thresh in enumerate(score_thresh_list):
    # Alter config
    version = first_version + i
    config['score_thresh'] = score_thresh
    config['version'] = version
    config['extractor_onnx_file'] = "cvnet-sg-v" + str(version) + ".onnx"
    config['pipeline_onnx_file'] = "pipeline-yolo-cvnet-sg-v" + str(version) + ".onnx"    
    PIPELINE_CONFIG_PATH = os.path.join("configs", "pipeline-yolo-cvnet-sg-v" + str(version))
    ROXFORD5K_DF = os.path.join(DATASETS_PATH, "roxford5k", "results", "df_roxford5k-v" + str(version) + ".pkl")
    ROXFORD5K_DESC = os.path.join(DATASETS_PATH, "roxford5k", "results", "desc_roxford5k-v" + str(version) + ".pkl")

    pipeline = Pipeline_Landmark_Detection(
            detector_file = config['detector_file'],
            extractor_onnx_file = config['extractor_onnx_file'],
            pipeline_onnx_file = config['pipeline_onnx_file'],
            image_dim = config['image_dim'],
            allowed_classes = config['allowed_classes'],
            score_thresh = config['score_thresh'],
            iou_thresh = config['iou_thresh'],
            scales = config['scales'],
            mean = config['mean'],
            std = config['std'],
            rgem_pr = config['rgem_pr'],
            rgem_size = config['rgem_size'],
            gem_p = config['gem_p'],
            sgem_ps = config['sgem_ps'],
            sgem_infinity = config['sgem_infinity'],
            eps = config['eps'],
            topk = topk,
            min_sim = min_sim,
            min_votes = min_votes,
            remove_inner_boxes = remove_inner_boxes,
            join_boxes = join_boxes
    )
    pipeline.to_json(PIPELINE_CONFIG_PATH)
    
    # Build/load roxford5k dataset
    df_result_ox, descriptors_final_ox, places_db_ox = pipeline.build_image_database(
        image_folder = ROXFORD5K_PATH,
        df_pickle_path = ROXFORD5K_DF,
        descriptor_pickle_path = ROXFORD5K_DESC,
        return_places_db = True,
        force_rebuild = force_rebuild,
        save_every = save_every,
        min_area = config['min_area'],
        min_sim = config['min_sim_db'],
        keep_full_img = config['keep_full_img']
    )
    
    # Evaluate roxford5k
    roxford5k_eval = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k')
    
    # Evaluate roxford5k with bbox
    roxford5k_eval_bbox = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k', use_bbox=True)
    
    results_eval = {
        'map_easy': roxford5k_eval['map_easy']*100,
        'mp@1_easy': roxford5k_eval['mpr_easy'][0]*100,
        'mp@5_easy': roxford5k_eval['mpr_easy'][1]*100,
        'mp@10_easy': roxford5k_eval['mpr_easy'][2]*100,
        'map_medium': roxford5k_eval['map_medium']*100,
        'mp@1_medium': roxford5k_eval['mpr_medium'][0]*100,
        'mp@5_medium': roxford5k_eval['mpr_medium'][1]*100,
        'mp@10_medium': roxford5k_eval['mpr_medium'][2]*100,
        'map_hard': roxford5k_eval['map_hard']*100,
        'mp@1_hard': roxford5k_eval['mpr_hard'][0]*100,
        'mp@5_hard': roxford5k_eval['mpr_hard'][1]*100,
        'mp@10_hard': roxford5k_eval['mpr_hard'][2]*100,
        'map_easy_bbox': roxford5k_eval_bbox['map_easy']*100,
        'mp@1_easy_bbox': roxford5k_eval_bbox['mpr_easy'][0]*100,
        'mp@5_easy_bbox': roxford5k_eval_bbox['mpr_easy'][1]*100,
        'mp@10_easy_bbox': roxford5k_eval_bbox['mpr_easy'][2]*100,
        'map_medium_bbox': roxford5k_eval_bbox['map_medium']*100,
        'mp@1_medium_bbox': roxford5k_eval_bbox['mpr_medium'][0]*100,
        'mp@5_medium_bbox': roxford5k_eval_bbox['mpr_medium'][1]*100,
        'mp@10_medium_bbox': roxford5k_eval_bbox['mpr_medium'][2]*100,
        'map_hard_bbox': roxford5k_eval_bbox['map_hard']*100,
        'mp@1_hard_bbox': roxford5k_eval_bbox['mpr_hard'][0]*100,
        'mp@5_hard_bbox': roxford5k_eval_bbox['mpr_hard'][1]*100,
        'mp@10_hard_bbox': roxford5k_eval_bbox['mpr_hard'][2]*100,
    }
    results = {**config, **results_eval}
    df = pd.DataFrame.from_dict([results])
    
    # Verificar si el archivo ya existe
    if os.path.exists(RESULTS_PATH):
        # Cargar el archivo existente
        df_saved = pd.read_csv(RESULTS_PATH)
    else:
        # Crear un DataFrame vacío con las claves de la nueva fila
        df_saved = pd.DataFrame(columns=df.keys())
    
    # Agregar la nueva fila
    df_saved = pd.concat([df_saved, df], ignore_index=True)
    
    # Guardar el archivo actualizado
    df_saved.to_csv(RESULTS_PATH, index=False)

Creando versión ONNX del preprocess
Instanciando el preprocessor
Creando versión ONNX del detector
Ultralytics 8.3.146  Python-3.11.11 torch-2.7.0+cpu CPU (Intel Core(TM) i5-9300HF 2.40GHz)
YOLOv8n summary (fused): 72 layers, 3,492,527 parameters, 0 gradients, 10.5 GFLOPs

PyTorch: starting from 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 605, 8400) (6.9 MB)

ONNX: starting export with onnx 1.17.0 opset 16...
ONNX: slimming with onnxslim 0.1.54...
ONNX: export success  4.2s, saved as 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.onnx' (13.5 MB)

Export complete (4.9s)
Results saved to C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models
Predict:         yolo predict task=detect model=C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_dete

Procesando imágenes:  76%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                   | 3859/5063 [44:50<13:59,  1.43it/s]


MemoryError: 